In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import torchvision
# Örnek bir transform tanımla (istediğin şekilde düzenleyebilirsin)
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Veri setini yükle
dataset = datasets.ImageFolder(root='./Kidney-Cancer', transform=transform)
BATCH_SIZE = 16
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size

train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])

# Eğitim verisini %10 doğrulama için ayır
val_size = int(0.1 * len(train_set))
train_size -= val_size  # Geriye kalan kısım eğitim verisi olacak

train_set, val_set = torch.utils.data.random_split(train_set, [train_size, val_size])

# DataLoader'ları oluştur (batch_size'i ihtiyacınıza göre ayarlayabilirsin)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)


C:\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT  
model = torchvision.models.efficientnet_b0(weights=weights).to(device)
model.classifier  = nn.Linear(1280,len(dataset.classes))
for param in model.parameters():
    param.requires_grad = False

for param in model.classifier .parameters():
    param.requires_grad = True

lossF = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001)


epochs = 15

for epoch in range(epochs):
    model.train().to(device)
    
    for inputs , labels in train_loader:
        inputs , labels = inputs.to(device),labels.to(device)
        optim.zero_grad()
        results = model(inputs)
        loss = lossF(results,labels)
        loss.backward()
        optim.step()
    
    print("Epoch : {}/{} , train loss : {}".format(epoch+1,epochs,loss.item()))
    
    
    model.eval()
    test_loss = 0
    acc = 0
    
    with torch.no_grad():
        for inputs,labels in val_loader:
            inputs , labels = inputs.to(device),labels.to(device)
            results = model(inputs)
            test_loss += lossF(results,labels).item()
            _,predicted = torch.max(results,1)
            acc += (predicted == labels).sum().item()
            
    test_loss  /= len(val_loader.dataset)
    accuracy  = acc/ len(val_loader.dataset)*100
    
    print(" Test loss : {} , test accuracy : {}".format(test_loss,accuracy))

Epoch : 1/15 , train loss : 0.2747979462146759
 Test loss : 0.005101214716477054 , test accuracy : 99.14285714285714
Epoch : 2/15 , train loss : 0.012309950776398182
 Test loss : 0.003097916603354471 , test accuracy : 99.42857142857143
Epoch : 3/15 , train loss : 0.12412998825311661
 Test loss : 0.0022589190091405597 , test accuracy : 99.42857142857143
Epoch : 4/15 , train loss : 0.008219779469072819
 Test loss : 0.002082240202331117 , test accuracy : 99.14285714285714
Epoch : 5/15 , train loss : 0.010182817466557026
 Test loss : 0.0021344169941065567 , test accuracy : 99.28571428571429
Epoch : 6/15 , train loss : 0.22395257651805878
 Test loss : 0.0017590189575483757 , test accuracy : 99.28571428571429
Epoch : 7/15 , train loss : 0.030475327745079994
 Test loss : 0.0011917756724038294 , test accuracy : 99.57142857142857
Epoch : 8/15 , train loss : 0.03022467903792858
 Test loss : 0.0011479157488793134 , test accuracy : 99.57142857142857
Epoch : 9/15 , train loss : 0.04634009301662445


In [3]:
torch.save(model.state_dict(), "transfer-learning-3.pth")